# Figure 10: Glacier signature uncertainty

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd

import os
from glob import glob
from oggm import utils

import plotly.express as px
import plotly.graph_objects as go
from   plotly.subplots import make_subplots

cl = px.colors.qualitative.D3
os.chdir('/home/rooda/Dropbox/Patagonia/')

## Data

In [ ]:
basins = gpd.read_file("GIS South/Basins_Patagonia_ice.shp")
basins = basins.set_index("ID")

basin_names = ["Yelcho", "Baker", "Santa Cruz", "Palena", "Grey", "Puelo", "Cisnes", "Aysen", "Pascua"] 
basins.loc[basins.basin_area > 5000, "Name"] = basin_names
basins = basins.dropna(subset = ['Name'])

In [ ]:
dict_replace = {"Metric": {"peak_water_year" : 'Timing', 
                           "peak_magnitude"  : 'Magnitude',
                            "frecuency"      : 'Frecuency',
                            "rate_of_change" : 'Rate of change',
                            "duration"       : 'Duration'}}

data = pd.read_csv("MS2 Results/variable_importance.csv", index_col = "ID")
data = data.replace(dict_replace)

# only major basins
data_main = data[data.index.isin(basins.index)]

In [ ]:
metrics = data.Metric.unique()

fig = make_subplots(rows=1, cols=5, horizontal_spacing = 0.01, vertical_spacing = 0.01, shared_yaxes = True,
                    subplot_titles = metrics, x_title='Permutation importance (RMSE)')

# a to e) Boxplot -----------------------------------------------------------------------------------------------------------
for x in range(0,5):
    data_x = data[data.Metric == metrics[x]]
    data_main_x = data_main[data_main.Metric == metrics[x]]
    fig.add_trace(go.Violin(x = data_x.Outline, name = "Geometry",  marker_color=cl[3], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_x.Volume,  name = "Volume",    marker_color=cl[1], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_x.Climate, name = "Climate",   marker_color=cl[2], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_x.GCM,     name = "GCM",       marker_color=cl[0], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_x.SSP,     name = "SSP",       marker_color=cl[4], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_x.BCM,     name = "BCM",       marker_color=cl[5], opacity=0.7, showlegend = False, points = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.Outline, name = "Geometry", marker_color=cl[3], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.Volume,  name = "Volume",   marker_color=cl[1], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.Climate, name = "Climate",  marker_color=cl[2], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.GCM,     name = "GCM",      marker_color=cl[0], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.SSP,     name = "SSP",      marker_color=cl[4], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)
    fig.add_trace(go.Violin(x = data_main_x.BCM,     name = "BCM",      marker_color=cl[5], opacity=0.9, line_color= "rgba(255,255,255,0)", fillcolor= "rgba(255,255,255,0)",  points="all", showlegend = False), row=1, col=x+1)


fig.update_traces(box_visible=True, width=0.7, meanline_visible=True, pointpos = 0.8, jitter = 0.2)
fig.update_xaxes(range = [-4,20], row = 1, col = 1)
fig.update_xaxes(range = [-30,150], row = 1, col = 2)
fig.update_xaxes(range = [-3,15], row = 1, col = 3)
fig.update_xaxes(range = [-1,10], row = 1, col = 4)
fig.update_xaxes(range = [-2,10], row = 1, col = 5)   
    
# general 
fig.update_layout(plot_bgcolor="rgba(213,213,213,0.4)")
fig.update_layout(width = 900, height = 450, template = "seaborn", autosize = False, margin = dict(l=0, r=0, b=50, t=30, pad=0, autoexpand=True))
fig.update_xaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=False, showline = True, ticks="outside")
fig.update_yaxes(griddash = "dot", gridcolor = "rgba(255,255,255,0.8)", zeroline=True,  showline = True, ticks="outside")

fig.write_image("/home/rooda/Dropbox/Patagonia/MS2 Results/Figure_10_signature.png", scale=4)
fig.show()    